# imports

In [ ]:
import pandas as pd
import numpy as np
import copy
import os
import sys
import shutil
import datetime
import pickle
import qplib as qp
from qplib import log, na, nk, num



# log

In [ ]:
import pandas as pd
import numpy as np
import copy
import os
import sys
import shutil
import datetime
import qplib as qp
from qplib import log
pd.set_option('display.max_columns', None)




log('trace: this is a trace message')
log('debug: this is a debug message')
log('info: this is an info message')
log('warning: this is a warning message')
log('error: this is an error message')

log()

# diff

In [ ]:

import pandas as pd
import numpy as np
import copy
import os
import datetime
import qplib as qp

from IPython.display import display
from ipywidgets import interact, widgets
from pandas.api.extensions import register_dataframe_accessor

from qplib.pandas import _format_df, get_dfs
from qplib.util import log, GREEN, RED, ORANGE, GREEN_LIGHT, RED_LIGHT, ORANGE_LIGHT
from qplib.types import _date, _na, _dict


df_new, df_old = get_dfs()


print('df_new:')
display(df_new)

print('df_old:')
display(df_old)

print('mode=new:')
display(qp.diff(df_new, df_old, mode='new'))

print('mode=new+:')
display(qp.diff(df_new, df_old, mode='new+'))

print('mode=old:')
display(qp.diff(df_new, df_old, mode='old'))

print('mode=mix:')
display(qp.diff(df_new, df_old, mode='mix'))


# merge


In [ ]:
import pandas as pd
import numpy as np
import copy
import os
import sys
import shutil
import datetime
import openpyxl
import qplib as qp
from qplib import log
pd.set_option('display.max_columns', None)

file = 'archive/NET_BM Study_01_export_2024-03-15.xlsx'
wb = openpyxl.load_workbook(file)
sheet_names = wb.sheetnames

crf = {}
for sheet_name in sheet_names:
    print(sheet_name)
    crf[sheet_name] = pd.read_excel(file, sheet_name=sheet_name)


In [ ]:
cols_flatten = [
    'MH_MedHist_MH_Term',
    'CM_ConMed_CM_Name',
    ]
# cols_flatten = None

summary = crf['VD_Visits_VD_Visit1']

for sheet_name in sheet_names[2:]:
    if sheet_name == 'VD_Visits_VD_Visit2':
        prefix = 'V2_'
    else:
        prefix = sheet_name[:3]
    summary = qp.merge(summary, crf[sheet_name], on='patientnr', flatten=cols_flatten, duplicates=True, prefix=prefix, verbosity=3)

# summary.to_excel('archive/merge4.xlsx', index=False)

# qlang

## format symbols

In [ ]:
import pandas as pd
import numpy as np
import copy
import os
import sys
import shutil
import datetime
import qplib as qp
from qplib import log
pd.set_option('display.max_columns', None)

def format_symbols():
    shutil.copy('qplib/data/symbols.xlsx', 'qplib/data/symbols_temp.xlsx')
    df = pd.read_excel('qplib/data/symbols_temp.xlsx', index_col=0)
    size = len(df.index)
    for i in range(2, size):
        row = df.index[i]
        for j in range(i+1):
            col = df.index[j]
            if row != col:
                df.loc[col, row] = df.loc[row, col]
    df.to_csv('qplib/data/symbols.csv')
    os.remove('qplib/data/symbols_temp.xlsx')
    return df

df = format_symbols()

# df


## performance analysis

In [ ]:
import pandas as pd
import numpy as np
import copy
import os
import sys
import shutil
import datetime
import qplib as qp
from qplib import log
pd.set_option('display.max_columns', None)
qp.qlang.VERBOSITY = 3


if not 'cards' in locals():
    cards = pd.read_csv('archive/cards.csv')


log(clear=True)

log('start')

cards.q(
    r"""
    power  %%>3  &&<5
    """
    )

log('stop')

logs = log().copy()
#milliseconds timestamps
logs['timestamp'] = logs['time'] - logs.loc[0, 'time']
logs['timestamp'] = logs['timestamp'].dt.total_seconds() * 1000


logs.q(
    r"""
    $ verbosity = 3
    $ diff = None

    text $width=500px $align=left
    
    %!=context  &!=time
    """
    )

# tests

In [ ]:

# #run tests in folder "tests" using pytest and create a test report
# !pytest tests --html=tests/test_report.html


In [ ]:
import pandas as pd
import numpy as np
import copy
import os
import sys
import shutil
import datetime
import qplib as qp
from qplib import log
pd.set_option('display.max_columns', None)


def get_df_simple():
    df = pd.DataFrame({
        'a': [-1, 0, 1],
        'b': [1, 2, 3]
        })
    return df

def get_df_simple_tagged():
    df = pd.DataFrame({
        'meta': ['', '', ''],
        'a': [-1, 0, 1],
        'b': [1, 2, 3]
        })
    df.index = pd.Index([3, 1, 2])
    return df


def get_df():
    df = pd.DataFrame({
        'ID': [10001, 10002, 10003, 20001, 20002, 20003, 30001, 30002, 30003, 30004, 30005],
        'name': ['John Doe', 'Jane Smith', 'Alice Johnson', 'Bob Brown', 'eva white', 'Frank miller', 'Grace TAYLOR', 'Harry Clark', 'IVY GREEN', 'JAck Williams', 'john Doe'],
        'date of birth': ['1995-01-02', '1990/09/14', '1985.08.23', '19800406', '05-11-2007', '06-30-1983', '28-05-1975', '1960Mar08', '1955-Jan-09', '1950 Sep 10', '1945 October 11'],
        'age': [-25, '30', np.nan, None, '40.0', 'forty-five', 'nan', 'unk', '', 'unknown', 35],
        'gender': ['M', 'F', 'Female', 'Male', 'Other', 'm', 'ff', 'NaN', None, 'Mal', 'female'],
        'height': [170, '175.5cm', None, '280', 'NaN', '185', '1', '6ft 1in', -10, '', 200],
        'weight': [70.2, '68', '72.5lb', 'na', '', '75kg', None, '80.3', '130lbs', '82', -65],
        'bp systole': ['20', 130, 'NaN', '140', '135mmhg', '125', 'NAN', '122', '', 130, '45'],
        'bp diastole': [80, '85', 'nan', '90mmHg', np.nan, '75', 'NaN', None, '95', '0', 'NaN'],
        'cholesterol': ['Normal', 'Highe', 'NaN', 'GOOD', 'n.a.', 'High', 'Normal', 'n/a', 'high', '', 'Normal'],
        'diabetes': ['No', 'yes', 'N/A', 'No', 'Y', 'Yes', 'NO', None, 'NaN', 'n', 'Yes'],
        'dose': ['10kg', 'NaN', '15 mg once a day', '20mg', '20 Mg', '25g', 'NaN', None, '30 MG', '35', '40ml']
        })
    return df


def get_df_tagged():
    df1 = get_df()
    df2 = pd.DataFrame('', index=df1.index, columns=['meta', *df1.columns])
    df2.iloc[:, 1:] = df1.loc[:, :]
    return df2


df = qp.get_df()
display(df)

  

param_pairs = [
    (r'name  %%is str;',                 df.loc[:, ['name']]),
    (r'name  %%!is str;',                df.loc[[], ['name']]),
    (r'name  %%is num;',                 df.loc[[], ['name']]),
    (r'name  %%!is num;',                df.loc[:, ['name']]),
    (r'name  %%is na;',                  df.loc[[], ['name']]),
    (r'name  %%!is na;',                 df.loc[:, ['name']]),

    (r'age   %%is int;',                 df.loc[[0,1,4,10], ['age']]),
    (r'age   %%strict is int;',          df.loc[[0,10], ['age']]),
    (r'age   %%is float;',               df.loc[[0,1,2,4,6,10], ['age']]),
    (r'age   %%strict is float;',        df.loc[[2], ['age']]),
    (r'age   %%is na;',                  df.loc[[2,3,6,8], ['age']]),

    (r'weight  %%is int;',               df.loc[[1,9,10], ['weight']]),
    (r'weight  %%strict is int;',        df.loc[[10], ['weight']]),
    (r'weight  %%is float;',             df.loc[[0,1,7,9,10], ['weight']]),
    (r'weight  %%strict is float;',      df.loc[[0], ['weight']]),
    (r'weight  %%is num;',               df.loc[[0,1,4,6,7,9,10], ['weight']]),
    (r'weight  %%strict is num;',        df.loc[[0,10], ['weight']]),
    (r'weight  %%is num;  &&!is na;',    df.loc[[0,1,7,9,10], ['weight']]),

    (r'height       %%is bool;',         df.loc[[6], ['height']]),
    (r'bp diastole  %%is bool;',         df.loc[[9], ['bp diastole']]),
    (r'diabetes     %%is bool;',         df.loc[[0,1,3,4,5,6,9,10], ['diabetes']]),
    (r'diabetes     %%strict is bool;',  df.loc[[], ['diabetes']]),

    (r'diabetes  %%is yn;',              df.loc[[0,1,3,4,5,6,9,10], ['diabetes']]),
    (r'diabetes  %%is na;  //is yn;',    df.loc[:, ['diabetes']]),
    (r'diabetes  %%is yes;',             df.loc[[1,4,5,10], ['diabetes']]),
    (r'diabetes  %%is no;',              df.loc[[0,3,6,9], ['diabetes']]),

    (r'cholesterol  %%is na;',           df.loc[[2,4,7,9], ['cholesterol']]),
    (r'age          %%is na;',           df.loc[[2,3,6,8], ['age']]),
    (r'age          %%strict is na;',           df.loc[[2,3], ['age']]),
    ]
def test(code, expected):
    df = get_df()
    temp = df.q(code)
    result = df.loc[temp.index, temp.columns]
    assert result.equals(expected), qp.diff(result, expected, output='str')

for code, expected in param_pairs:
    test(code, expected)




# test coverage

In [ ]:
import pandas as pd
import numpy as np
import copy
import os
import sys
import shutil
import pytest
import datetime
import qplib as qp
from qplib import log
pd.set_option('display.max_columns', None)

get_df =  qp.get_df
df = get_df()

def check_message(expected_message):
    logs = qp.log()
    logs['text_full'] = logs['level'] + ': ' + logs['text']
    log_texts = logs['text_full'].to_list()
    assert expected_message in logs['text_full'].values, f'did not find expected message: {expected_message}\nin logs:\n{log_texts}'



In [ ]:


# display(result)
# display(expected)

# display(set(result.columns) - set(expected.columns))
# display(set(expected.columns) - set(result.columns))
# display(set(result.index) - set(expected.index))
# display(set(expected.index) - set(result.index))
# display('result dtypes:', result.dtypes)
# display('expected dtypes:', expected.dtypes)

# display(result == expected)
# print(result.equals(expected))


# temp

In [ ]:
import pandas as pd
import numpy as np
import copy
import os
import sys
import shutil
import datetime
import qplib as qp
from qplib import log
pd.set_option('display.max_columns', None)

def get_dfs():
    df1 = pd.DataFrame({
        'uid': [1, 2, 3],
        'age': [42, 17, 55],
        'IC': ['y', 'n', 'y'],
        })

    df2 = pd.DataFrame({
        'uid': [1, 1, 2],
        'age': [42, 42, 17],
        'term': ['headache', 'nausea', 'headache'],
        'start': [datetime.date(2023, 1, 1), None, datetime.date(2021, 12, 3)],
        })
    
    df3 = pd.DataFrame({
        'uid': [1, 2, 2],
        'age': [42, 17, 17],
        'term': ['mexalen', 'aspirin', 'ibuprofen'],
        'dose': ['1mg', None, 3],
        })

    return df1, df2, df3

df1, df2, df3 = get_dfs()